# 导入模块

In [10]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text, MetaData, Table
from sqlalchemy.exc import OperationalError, SQLAlchemyError
import warnings
warnings.filterwarnings('ignore')
from urllib.parse import quote_plus
#   pip install sqlalchemy==2.0.20 需要2.0版本
pd.set_option('display.max_columns', None)


In [16]:
# def create_engines():
#     # 创建数据库连接引擎
#     # 请根据你的实际情况修改以下参数
#     try:
#         engine = create_engine('mysql+pymysql://ylc:digua2025@)@%@rm-wz95u72vb6922ju02bo.mysql.rds.aliyuncs.com/db_digua_business?charset=utf8mb4')
#         with engine.connect() as connection:
#             print("数据库连接成功！")
#     except OperationalError as e:
#         print(f"连接失败：{e}")
#         print("请检查：1) 用户名/密码 2) 主机地址 3) 端口号 4) 数据库是否存在 5) MySQL服务是否运行")
#     return engine


def create_engines():
    # 创建数据库连接引擎
    try:
        # 数据库连接参数配置
        db_username = 'ylc'           # 数据库用户名
        db_password = 'digua2025@)@%'     # 数据库密码
        db_host = 'rm-wz95u72vb6922ju02bo.mysql.rds.aliyuncs.com'  # 数据库主机地址
        db_port = '3306'              # 数据库端口号
        db_name = 'db_digua_business' # 数据库名称
        
        # 对密码中的特殊字符进行URL编码
        encoded_password = quote_plus(db_password)
        
        # 构建连接字符串 - 使用编码后的密码
        connection_string = f'mysql+pymysql://{db_username}:{encoded_password}@{db_host}:{db_port}/{db_name}?charset=utf8mb4'
        
        # 创建数据库引擎
        engine = create_engine(connection_string)
        
        # 测试连接
        with engine.connect():
            print("数据库连接成功！")
        
        return engine
        
    except OperationalError as e:
        print(f"连接失败：{e}")
        print("请检查：")
        print(f"1) 数据库用户名: {db_username}")
        print("2) 数据库密码: ***** (已进行URL编码处理)")
        print(f"3) 数据库主机地址: {db_host}")
        print(f"4) 数据库端口号: {db_port}")
        print(f"5) 数据库名称: {db_name}")
        print("6) 网络连接是否正常")
        print("7) 数据库服务是否运行")
    except Exception as e:
        print(f"发生其他错误：{e}")
    
    return None

In [26]:


def add_columns(table_name, new_column, column_type, default_value, remark):
    '''
    table_name：表名
    new_column：新字段名
    column_type：字段数据类型
    default_value：默认值
    remark：备注
    '''
    engine = create_engines()
    
    try:
        # 获取数据表中字段名，并判断需要写入的字段是否在数据表中
        metadata = MetaData()
        # 反射表结构
        table = Table(table_name, metadata, autoload_with=engine)
        # 获取字段名列表
        column_names = [column.name for column in table.columns]
        
        with engine.connect() as connection:
            # 判断需要写入的字段名是否在字段名列表中
            if new_column in column_names:
                # 字段已存在，先删除该字段
                drop_query = text(f"""
                    ALTER TABLE {table_name}
                    DROP COLUMN {new_column}
                """)
                connection.execute(drop_query)
                print(f"已删除原有字段 {new_column}")
            
            # 执行 ALTER TABLE 语句添加新字段
            # 判断字段类型是否为字符串类型，从而决定是否添加字符集和排序规则
            if column_type.lower() in ['varchar', 'char', 'text', 'longtext', 'mediumtext', 'tinytext']:
                # 字符串类型字段需要指定字符集和排序规则
                alter_query = text(f"""
                    ALTER TABLE {table_name}
                    ADD COLUMN {new_column} {column_type} CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci DEFAULT {default_value} COMMENT '{remark}'
                """)
            else:
                # 数值类型字段不需要字符集和排序规则
                alter_query = text(f"""
                    ALTER TABLE {table_name}
                    ADD COLUMN {new_column} {column_type} DEFAULT {default_value} COMMENT '{remark}'
                """)
            
            connection.execute(alter_query)
            print(f"成功添加字段 {new_column} 到表 {table_name}")
            
    except SQLAlchemyError as e:
        print(f"添加字段失败: {e}")
    finally:
        # 关闭数据库连接
        engine.dispose()

In [28]:
# # 表名，字段名，类型，默认值，备注
# # add_columns('test', 'M7', 'varchar(255)', '', 'M7')
# table_list = ['Vintage30_order', 'Vintage60_order', 'Vintage90_order', 'Vintage30_purchase_amount', 'Vintage60_purchase_amount', 'Vintage90_purchase_amount']
# # 为每个表添加MOB36字段
# for table_name in table_list:
#     add_columns(
#         table_name=table_name,
#         new_column='sort36',
#         column_type='varchar(255)',
#         default_value='NULL',
#         remark='sort36'
#     )

数据库连接成功！
已删除原有字段 sort36
成功添加字段 sort36 到表 Vintage30_order
数据库连接成功！
已删除原有字段 sort36
成功添加字段 sort36 到表 Vintage60_order
数据库连接成功！
已删除原有字段 sort36
成功添加字段 sort36 到表 Vintage90_order
数据库连接成功！
已删除原有字段 sort36
成功添加字段 sort36 到表 Vintage30_purchase_amount
数据库连接成功！
已删除原有字段 sort36
成功添加字段 sort36 到表 Vintage60_purchase_amount
数据库连接成功！
已删除原有字段 sort36
成功添加字段 sort36 到表 Vintage90_purchase_amount


In [29]:
# 写入数据
def insert_data(df, table_name):
    '''
    df：需要写入的数据集
    table_name：需要写入的表名
    '''
    engine = create_engines()
    try:
        # 查询数据库，判断是否有数据，有数据则删除数据在执行插入数据操作
        try:
            with engine.begin() as connection:
                # 查询数据库是否有数据
                result = connection.execute(text(f"SELECT EXISTS(SELECT 1 FROM {table_name} LIMIT 1)")).scalar()
                if result:
                    # 删除表数据
                    connection.execute(text(f"""DELETE FROM {table_name}"""))
                    print(f"成功删除表 {table_name} 的数据")
                else:
                    print(f"表 {table_name} 没有数据 请插入数据")
        except SQLAlchemyError as e:
            print(f"删除数据失败: {e}")
        # 将 DataFrame 写入数据库
        # name：表名
        # con：数据库连接
        # if_exists：如果表已存在的处理方式（'fail'：失败，'replace'：替换，'append'：追加）
        # index：是否将索引写入数据库
        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='append',
            index=False
        )
        print("数据成功写入数据库！")
        
    except Exception as e:
        print(f"写入数据时出错：{e}")
        
    finally:
        # 关闭数据库连接
        engine.dispose()


# 读取数据

In [37]:
pare = pd.ExcelFile('F:/租后表分析数据_ld/写入数据库/迪瓜租机数据 to 客商_存数据库_20250908.xlsx')
# 获取Excel文件中所有工作表的名称列表
sheet_names = pare.sheet_names #['业务量及通过率', '余额衰减', 'vintage（金额比例）', '每月截面数据', '首逾率', '迁徙率']
# '余额衰减'，header=1 表示使用第二行作为列名
df1 = pare.parse(sheet_names[1], header=1)
# 'vintage（金额比例）'
df2 = pare.parse(sheet_names[2], header=1)
df2 = df2.iloc[:, 1:]
# '每月截面数据'
df3 = pare.parse(sheet_names[3], header=1)
# '首逾率'
df4 = pare.parse(sheet_names[4], header=2)
df4 = df4.iloc[:, 1:]
# '迁徙率'
df5 = pare.parse(sheet_names[5], header=1)

# 余额衰减

In [ ]:
# df1_1 = df1.iloc[:38, :39]
# df1_1

In [38]:
def update_col1(df):
    df_col_list = df.columns.to_list()
    
    for col in df_col_list:
        col_new = col.split('.')[0]
        # 将原来的列名 col 改为 col_new，inplace=True 表示直接在原DataFrame上修改。
        df.rename(columns={col: f'{col_new}'}, inplace=True) 
    df.rename(columns={'下单月份': 'search_time', '出库订单数': 'out_order_number', '采购金额': 'purchase_amount'}, inplace=True)
    # df.loc[:, 'search_time'] = df.search_time.apply(lambda x: str(x).split('.')[0]+'-'+(str(x).split('.')[1] if str(x).split('.')[1]!='1' else '10'))
    df.loc[:, 'search_time'] = df.search_time.apply(
        lambda x: x if pd.isna(x) else
        str(x).split('.')[0] + '-' + (str(x).split('.')[1] if str(x).split('.')[1] != '1' else '10')
        if len(str(x).split('.')) > 1 else str(x)
    )
    return df

# 余额衰减 订单口径
# 从 df1 中提取前38行、前39列的数据,
df1_1 = df1.iloc[:38, :39]
df1_1 = update_col1(df1_1)

df1_2 = df1.iloc[42:80, :39]
df1_2 = update_col1(df1_2)

df1_3 = df1.iloc[84:122, :39]
df1_3 = update_col1(df1_3)

df1_4 = df1.iloc[126:164, :39]
df1_4 = update_col1(df1_4)
# 金额口径
df1_5 = df1.iloc[:38, 40:80]
df1_5 = update_col1(df1_5)

df1_6 = df1.iloc[42:80, 40:80]
df1_6 = update_col1(df1_6)

df1_7 = df1.iloc[84:122, 40:80]
df1_7 = update_col1(df1_7)

df1_8 = df1.iloc[126:164, 40:80]
df1_8 = update_col1(df1_8)

df_list = [df1_1, df1_2, df1_3, df1_4, df1_5, df1_6, df1_7, df1_8]
table_list = ['mob_order_all','mob_order_12m','mob_order_18m','mob_order_24m', 'mob_purchase_amount_all', 'mob_purchase_amount_12m', 'mob_purchase_amount_18m','mob_purchase_amount_24m']
for idx, table_name in enumerate(table_list):
    insert_data(df_list[idx], table_name)


数据库连接成功！
成功删除表 mob_order_all 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 mob_order_12m 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 mob_order_18m 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 mob_order_24m 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 mob_purchase_amount_all 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 mob_purchase_amount_12m 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 mob_purchase_amount_18m 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 mob_purchase_amount_24m 的数据
数据成功写入数据库！


In [ ]:
# insert_data(df1_4, 'mob_order_24m')
df2.iloc[80:114, 37:71]

# vintage（金额比例）

In [32]:
# # vintage（金额比例）
# def update_col2(df2):
#     # 修改列名
#     df2_col_list = df2.columns.to_list()
#     for col in df2_col_list:
#         if col=='Month' or col=='Month.1':
#             df2.rename(columns={f'{col}': 'search_time'}, inplace=True)
#         else:
#             col_new = str(col).split('.')[0]
#             # 按.分割取第一部分，并加上前缀'sort'，inplace=True 表示直接在原DataFrame上修改。
#             df2.rename(columns={col: f'sort{col_new}'}, inplace=True)
#             # 将列中的数值转换为百分比格式，保留两位小数，NaN值保持不变
#             df2.loc[:, f'sort{col_new}'] = df2[f'sort{col_new}'].apply(lambda x: x if pd.isna(x) else format(x, '.2%'))
#     df2.loc[:, 'search_time'] = df2.search_time.apply(lambda x: str(x).split('.')[0]+'-'+(str(x).split('.')[1] if str(x).split('.')[1]!='1' else '10'))
#     return df2
# # 取到固定范围的数据 订单维度
# df2_1 = df2.iloc[:36, :36]
# df2_1 = update_col2(df2_1)
# df2_2 = df2.iloc[40:75, :35]
# df2_2.columns = df2.iloc[39, :35]
# df2_2 = update_col2(df2_2)
# df2_3 = df2.iloc[80:114, :34]
# df2_3.columns = df2.iloc[79, :34]
# df2_3 = update_col2(df2_3)
# # 金额维度
# df2_4 = df2.iloc[:36, 37:73]
# df2_4 = update_col2(df2_4)
# df2_5 = df2.iloc[40:75, 37:72]
# df2_5.columns = df2.iloc[39, 37:72]
# df2_5 = update_col2(df2_5)
# df2_6 = df2.iloc[80:114, 37:71]
# df2_6.columns = df2.iloc[79, 37:71]
# df2_6 = update_col2(df2_6)
# # 写入数据库
# df2_list = [df2_1, df2_2, df2_3, df2_4, df2_5, df2_6]
# table_list2 = ['Vintage30_order', 'Vintage60_order', 'Vintage90_order', 'Vintage30_purchase_amount', 'Vintage60_purchase_amount', 'Vintage90_purchase_amount']
# for idx, table_name in enumerate(table_list2):
#     insert_data(df2_list[idx], table_name)

数据库连接成功！
成功删除表 Vintage30_order 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 Vintage60_order 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 Vintage90_order 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 Vintage30_purchase_amount 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 Vintage60_purchase_amount 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 Vintage90_purchase_amount 的数据
数据成功写入数据库！


# 每月截面数据

In [ ]:
df3

In [39]:
def update_col3(df3, key=None):
    # key 用来判断是否需要将数值类型修改为int类型
    # 获取第一行之后的数据
    df3 = df3.iloc[1:, :]
    df3_col_list = df3.columns.to_list()
    # 循环列名，对列名和数据进行修改
    for col in df3_col_list:
        col_new = col.split('.')[1]
        if col_new=='正常':
            col_new = 'C'
        elif col_new=='M6+':
            col_new = 'M6_Plus'
        
        df3.rename(columns={col: f'{col_new}'}, inplace=True)
        if key=='int':
            df3.loc[:, col_new] = df3[col_new].apply(lambda x: x if pd.isna(x) else int(x))
    df3.reset_index(names=['search_time'], inplace=True)
    df3.loc[:, 'search_time'] = df3.search_time.apply(lambda x: str(x).split('.')[0]+'-'+(str(x).split('.')[1] if str(x).split('.')[1]!='1' else '10'))
    return df3
# '每月截面数据' 订单维度
df3_new1 = df3.iloc[:8, :].T
df3_new1.columns=df3_new1.iloc[0, :]
df3_new1 = update_col3(df3_new1, 'int')

# 金额维度
df3_new2 = df3.iloc[11:, :].T
df3_new2.columns=df3_new2.iloc[0, :]
df3_new2 = update_col3(df3_new2)

# 写入数据
df3_list = [df3_new1, df3_new2]
table_list3 = ['rental_status_order', 'rental_status_purchase_amount']
for idx, table_name in enumerate(table_list3):
    insert_data(df3_list[idx], table_name)
# df3_new2

数据库连接成功！
成功删除表 rental_status_order 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 rental_status_purchase_amount 的数据
数据成功写入数据库！


# 首逾率

In [ ]:
df4.iloc[:38, :]

In [34]:
# def update_col4(df4):
#     df4_col_list = df4.columns.to_list()
#     for col in df4_col_list:
#         # 第一、二列列名需修改
#         if col=='Unnamed: 1':
#             col_new = 'search_time'
#         elif col=='Unnamed: 2':
#             col_new = 'order_number'
#         elif '%' in col:
#             col_new = col.strip('%').strip('+')+'_rate_plus'
#         else:
#             col_new = col.strip('+')+'_plus'
#         df4.rename(columns={col: col_new}, inplace=True)
#         if '_rate_plus' in col_new:
#             df4.loc[:, col_new] = df4[col_new].apply(lambda x: x if pd.isna(x) or type(x)==str else format(x, '.2%'))
#     # df4.iloc[:-1, 0] = df4.iloc[:-1, :].search_time.apply(lambda x: str(x).split('.')[0]+'-'+(str(x).split('.')[1] if str(x).split('.')[1]!='1' else '10'))
#     return df4
# # '首逾率' 订单维度
# df4_1 = df4.iloc[:38, :]
# df4_1 = update_col4(df4_1)

# # 金额维度
# df4_2 = df4.iloc[43:, :]
# df4_2 = update_col4(df4_2)

# # 写入数据
# df4_list = [df4_1, df4_2]
# table_list4 = ['overdue_fst_order', 'overdue_fst_purchase_amount']
# for idx, table_name in enumerate(table_list4):
#     insert_data(df4_list[idx], table_name)

数据库连接成功！
成功删除表 overdue_fst_order 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 overdue_fst_purchase_amount 的数据
数据成功写入数据库！


# 迁徙率

In [ ]:
df5.iloc[9:, :]

In [40]:
def update_col5(df5):
    df5_col_list = df5.columns.to_list()
    for col in df5_col_list:
        df5.loc[:, col] = df5[col].apply(lambda x: x if pd.isna(x) else format(x, '.2%'))
    df5.reset_index(names=['search_time'], inplace=True)
    df5.loc[:, 'search_time'] = df5.search_time.apply(lambda x: str(x).split('.')[0]+'-'+(str(x).split('.')[1] if str(x).split('.')[1]!='1' else '10'))
    return df5
# '迁徙率' 订单口径
df5_1 = df5.iloc[:6, :].T
df5_1.columns=df5_1.iloc[0, :]
df5_1 = df5_1.iloc[1:, :]
df5_1 = update_col5(df5_1)

# 金额口径
df5_2 = df5.iloc[9:, :].T
df5_2.columns=df5_2.iloc[0, :]
df5_2 = df5_2.iloc[1:, :]
df5_2 = update_col5(df5_2)

# 写入数据库
df5_list = [df5_1, df5_2]
table_list5 = ['rollrate_order', 'rollrate_purchase_amount']
for idx, table_name in enumerate(table_list5):
    insert_data(df5_list[idx], table_name)


数据库连接成功！
成功删除表 rollrate_order 的数据
数据成功写入数据库！
数据库连接成功！
成功删除表 rollrate_purchase_amount 的数据
数据成功写入数据库！
